# Init Vault

In [ ]:
%%bash
# gcloud auth login
eval "$(terraform output -raw cluster1_init_remote)"

# If you want to save time and get the token directly uncomment
#echo "VAULT TOKEN: $VAULT_TOKEN"
#echo "VAULT ADDR: $VAULT_ADDR"

# Init Boundary

In [2]:
%%bash
mkdir init_boundary
cat << EOF > ./init_boundary/main.tf 
# https://developer.hashicorp.com/boundary/docs/install-boundary/initialize#create-your-first-login-account
provider "boundary" {
  addr             = "$(terraform output -raw fqdn_443)"
  tls_insecure     = true
  recovery_kms_hcl = <<EOT
    kms "gcpckms" {
    purpose     = "recovery"
    key_ring    = "$(terraform output -raw key_ring)"
    crypto_key  = "$(terraform output -raw crypto_key)"
    project     = "$(terraform output -raw project)"
    region      = "$(terraform output -raw location)"
    }
EOT
}

resource "boundary_scope" "org" {
  scope_id    = "global"
  name        = "global"
  description = "Organization scope"

  auto_create_admin_role   = false
  auto_create_default_role = false
}

resource "boundary_scope" "project" {
  name                     = "project"
  description              = "My first project"
  scope_id                 = boundary_scope.org.id
  auto_create_admin_role   = false
  auto_create_default_role = false
}

resource "boundary_auth_method" "password" {
  name        = "Password auth method"
  description = "Password auth method"
  type        = "password"
  scope_id    = boundary_scope.org.id
}

resource "boundary_account_password" "myuser" {
  name           = "admin"
  description    = "User account for adminr"
  login_name     = var.boundary_user
  password       = var.boundary_password
  auth_method_id = boundary_auth_method.password.id
}

resource "boundary_user" "myuser" {
  name        = "admin"
  description = "Initial Admin User"
  account_ids = [boundary_account_password.myuser.id]
  scope_id    = boundary_scope.org.id
}

resource "boundary_role" "org_admin" {
  scope_id        = "global"
  grant_scope_ids = [boundary_scope.org.id]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]
}

resource "boundary_role" "project_admin" {
  scope_id        = boundary_scope.org.id
  grant_scope_ids = [boundary_scope.project.id]
  grant_strings = [
    "ids=*;type=*;actions=*"
  ]
  principal_ids = [boundary_user.myuser.id]

}

EOF

cat << EOF > ./init_boundary/variables.tf 
variable "boundary_user" {
  description = "Initial Boundary Admin User"
  type        = string
}

variable "boundary_password" {
  description = "Password for Boundary Admin"
  type        = string
}
EOF

cat << EOF > ./init_boundary/variables.tfvars
boundary_user        = "admin"            #Use your own
boundary_password    = "SuperSecret123!"  #Use your own
EOF

In [ ]:
%%bash
cd init_boundary
terraform init
terraform apply --auto-approve --var-file=variables.tfvars

## To destroy configuration

In [ ]:
%%bash
cd init_boundary
terraform destroy --auto-approve --var-file=variables.tfvars